In [7]:
from curl_cffi import requests
session = requests.Session(impersonate="firefox")
stock = yf.Ticker('AAPL')
historical_data = stock.history(period='1y')

YFRateLimitError: Too Many Requests. Rate limited. Try after a while.

In [20]:
import requests
import pandas as pd

# Free API key required from alphavantage.co
API_KEY = '3GDHNN74MX8YBM97'#'your_api_key_here'

def get_stock_data_alphavantage(symbol):
    url = f'https://www.alphavantage.co/query'
    params = {
        'function':'REALTIME_OPTIONS', #'TIME_SERIES_DAILY',
        'symbol': symbol,
        'apikey': API_KEY,
        'require_greeks':True
        #'outputsize': outputsize
    }
    
    response = requests.get(url, params=params)
    data = response.json()
    return data    
    # if 'Time Series (Daily)' in data:
    #     df = pd.DataFrame(data['Time Series (Daily)']).T
    #     df.index = pd.to_datetime(df.index)
    #     df = df.astype(float)
    #     return df
    # else:
    #     print("Error:", data.get('Error Message', 'Unknown error'))
    #     return None

# Usage
stock_data = get_stock_data_alphavantage('AAPL')


In [29]:
#create instance
ref_FOC = FOC()

#fetch current stock price for AAPL
stock_price = ref_FOC.get_stock_price("AAPL")


Exception in thread Thread-36 (process_queue):
Traceback (most recent call last):
  File "/Users/williamxu/anaconda3/lib/python3.11/site-packages/urllib3/connection.py", line 198, in _new_conn
    sock = connection.create_connection(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/williamxu/anaconda3/lib/python3.11/site-packages/urllib3/util/connection.py", line 60, in create_connection
    for res in socket.getaddrinfo(host, port, family, socket.SOCK_STREAM):
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/williamxu/anaconda3/lib/python3.11/socket.py", line 962, in getaddrinfo
    for res in _socket.getaddrinfo(host, port, family, type, proto, flags):
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
socket.gaierror: [Errno 8] nodename nor servname provided, or not known

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/Users/williamxu/anaconda3/li

In [34]:
options_chain = ref_FOC.get_options_chain_greeks("AAPL","2023-10-06",'CALL')

ValueError: Invalid json keys.

In [36]:

# Options Chain using custom enum
options_chain = ref_FOC.get_options_chain("AAPL", "2024-10-06",'CALL')

ValueError: Invalid json keys.

In [37]:
ref_FOC.create_quote_stream_options_chain(5,"AMC","2023-10-20","CALL",result_callback=example_result_callback)


Exception in thread Thread-44 (_run):
Traceback (most recent call last):
  File "/Users/williamxu/anaconda3/lib/python3.11/site-packages/FOC/main.py", line 116, in json_extract_node
    json_data = json_data[key]
                ~~~~~~~~~^^^^^
TypeError: 'NoneType' object is not subscriptable

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/williamxu/anaconda3/lib/python3.11/threading.py", line 1045, in _bootstrap_inner
    self.run()
  File "/Users/williamxu/anaconda3/lib/python3.11/site-packages/ipykernel/ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "/Users/williamxu/anaconda3/lib/python3.11/threading.py", line 982, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/williamxu/anaconda3/lib/python3.11/site-packages/FOC/intervalrunner.py", line 23, in _run
    result = self.target_func(*self.func_args)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/wil

In [30]:
stock_price

,price,nlsShareVolume,nlsTime,nlsVolume,previousClose,todayHighLow,fiftyTwoWeekHighLow
0,201.36,144.0,16:00:01,"46,721,776",$202.09,$202.72/$199.70,$260.1/$169.2101


In [32]:
def example_result_callback(result):
    print("Result from target function:", result)

In [45]:
d=ref_FOC.create_quote_stream_options_price_data(2,'AMC---250523C00004000',result_callback=example_result_callback)


Result from target function: label                 Last Sale   Net Day High Day Low Volume Prev Close  \
AMC---250523C00004000                                                      
value                      0.02  0.01     0.03    0.01   9833       0.01   

label                  Open Open Interest Tick   Bid  ... Contract Low Market  \
AMC---250523C00004000                                 ...                       
value                  0.01          7851       0.01  ...         0.01  OPRAC   

label                    Delta    Gamma      Rho     Theta     Vega   Impvol  \
AMC---250523C00004000                                                          
value                  0.07712  0.31683  0.00001  -0.03302  0.00025  2.69114   

label                            timestamp        contract_symbol  
AMC---250523C00004000                                              
value                  2025-05-22 19:30:00  AMC---250523C00004000  

[1 rows x 24 columns]
Result from target function: l

In [56]:
d=ref_FOC.create_quote_stream_options_price_data(2,'TSLA---250523C00360000',result_callback=example_result_callback)


Exception in thread Thread-65 (_run):
Traceback (most recent call last):
  File "/Users/williamxu/anaconda3/lib/python3.11/site-packages/FOC/main.py", line 116, in json_extract_node
    json_data = json_data[key]
                ~~~~~~~~~^^^^^
TypeError: 'NoneType' object is not subscriptable

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/williamxu/anaconda3/lib/python3.11/threading.py", line 1045, in _bootstrap_inner
    self.run()
  File "/Users/williamxu/anaconda3/lib/python3.11/site-packages/ipykernel/ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "/Users/williamxu/anaconda3/lib/python3.11/threading.py", line 982, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/williamxu/anaconda3/lib/python3.11/site-packages/FOC/intervalrunner.py", line 23, in _run
    result = self.target_func(*self.func_args)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/wil

In [58]:
d=ref_FOC.create_quote_stream_options_price_data(2,'TGT---250523C00100000',result_callback=example_result_callback)


Result from target function: label                 Last Sale    Net Day High Day Low Volume Prev Close  \
TGT---250523C00100000                                                       
value                      0.06  -0.02     0.15    0.04   1164       0.08   

label                 Open Open Interest Tick   Bid  ... Contract Low Market  \
TGT---250523C00100000                                ...                       
value                  0.1          4587       0.05  ...         0.07  OPRAC   

label                    Delta    Gamma      Rho     Theta     Vega   Impvol  \
TGT---250523C00100000                                                          
value                  0.05003  0.03555  0.00013  -0.15009  0.00513  0.58416   

label                            timestamp        contract_symbol  
TGT---250523C00100000                                              
value                  2025-05-22 19:39:00  TGT---250523C00100000  

[1 rows x 24 columns]
Result from target function: l

In [ ]:
d=ref_FOC.create_quote_stream_options_price_data(2,'AAPL--250523C00155000',result_callback=example_result_callback)

#AAPL250523C00155000

Result from target function: label                 Last Sale Net Day High Day Low Volume Prev Close Open  \
AAPL--250523C00155000                                                         
value                     57.61   -      N/A     N/A      0      57.61  N/A   

label                 Open Interest Tick    Bid  ... Contract Low Market  \
AAPL--250523C00155000                            ...                       
value                            90       45.55  ...        28.58  OPRAC   

label                    Delta    Gamma      Rho     Theta     Vega   Impvol  \
AAPL--250523C00155000                                                          
value                  0.95121  0.00302  0.00395  -1.71265  0.01064  3.18287   

label                            timestamp        contract_symbol  
AAPL--250523C00155000                                              
value                  2025-05-22 19:43:00  AAPL--250523C00155000  

[1 rows x 24 columns]
Result from target function: label  

In [26]:
from FOC import FOC

# Initialize FOC
ref_FOC = FOC()

# Get stock price
stock_price = ref_FOC.get_stock_price("AAPL")

# Get options chain (price only)
options_chain = ref_FOC.get_options_chain("AAPL", "2023-11-10", "CALL")

# Get options chain with Greeks
options_with_greeks = ref_FOC.get_options_chain_greeks("AAPL", "2023-11-10", "CALL")

# Get single option data (correct method)
contract_symbol = ref_FOC.get_contract_symbol("AAPL", '2023-11-10', 'CALL', 175.0)
single_option_data = ref_FOC.get_options_price_data(contract_symbol)

# Get expiration dates
expiration_dates = ref_FOC.get_expiration_dates("AAPL")


ValueError: Invalid json keys.

In [2]:
list_name='ftse250'

#list_name='SP500'
# ending=''
filel=open('../gathering/data/'+list_name+'.txt','a+')
filel.seek(0)
stocklist=filel.read().splitlines()
filel.close()

In [ ]:
res=stock_list_annotation(stocklist,client,'UK')

3IN
3i Infrastructure is a listed investment company that invests in infrastructure businesses, mainly in Europe and Asia, focusing on utilities, transportation and social infrastructure.

16.726508546292564
FOUR
4imprint Group plc is a direct marketer and distributor of promotional products like apparel, stationery, and technology, primarily in North America and the UK & Ireland.

7.881796072725105
ABDN
abrdn plc is a global investment company managing investments and providing financial planning solutions through its Investments, Adviser, and Interactive Investor segments.

3.208990421756395
ALFA
Alfa Financial Software Holdings PLC provides software and services, including its Alfa Systems platform, to the auto and equipment finance industry globally.

13.896374708942126
ATT
Allianz Technology Trust is a UK investment trust aiming for long-term capital growth by investing in quoted technology companies worldwide.

13.394699904187146
ALPH
Alpha Group International PLC is a UK-based f

ERROR:yfinance:$BHME.L: possibly delisted; no price data found  (period=1d)


BHME
N/A Business Services offers outsourced business support, including virtual PA, financial management, and event management, to small and medium-sized businesses.

49.09458261937652
BYG
Big Yellow Group PLC is a UK-based company that acquires, owns, and manages self-storage facilities.

16.556437120434015
BRGE
BlackRock Greater Europe Ord is a UK investment trust focused on achieving capital growth through a portfolio of European companies, including large, mid, and small capitalization businesses.

22.905477384678793
BRSC
BlackRock Smaller Companies Trust PLC invests in smaller UK companies listed on the London Stock Exchange to achieve long-term capital growth for shareholders.

13.771011322767135
THRG
BlackRock Throgmorton Trust plc invests primarily in UK smaller and mid-capitalization companies traded on the London Stock Exchange, seeking long-term capital growth and an attractive total return for shareholders.

36.16239913230656
BRWM
BlackRock World Mining Trust aims to provi

ERROR:yfinance:$JEO.L: possibly delisted; no price data found  (period=1d) (Yahoo error = "No data found, symbol may be delisted")


Unfortunately, I cannot provide a summary of the business of a UK company called "N/A" because "N/A" typically means "Not Applicable" or that the information is not available.
14.399910470639387


...

Please provide more context. I need the company's name to summarize its business. "N/A" means "Not Applicable" and doesn't give me any information.

24.275974171858977
MRCM


ERROR:yfinance:$MRCM.L: possibly delisted; no price data found  (period=1d) (Yahoo error = "No data found, symbol may be delisted")
ERROR:yfinance:404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/MRCM.L?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=MRCM.L&crumb=rxEqWkrFUhk


I am sorry, I cannot summarise the business of UK company N/A because "N/A" does not provide enough information to determine the company's activities. To provide a useful summary, I would need a valid company name or registration number.

18.652325769968225
MRCH
Merchants Trust Ord is a UK investment company aiming to provide above-average income and long-term capital growth by investing mainly in higher-yielding large UK companies.

19.39807964535634
MTO
Mitie Group PLC is a UK-based facilities management and professional services company providing services to both public and private sectors.

19.703251009458945
MAB
Mitchells & Butlers plc operates a wide range of pubs, bars, and restaurants across the UK and Germany.

33.16577187636594
MTO
Mitie Group PLC is a British strategic outsourcing and energy services company, providing facilities and property management, infrastructure consultancy, and technical services.

10.32475504041881
MONY
MONY Group PLC operates a tech-led savings pla

RHIM
RHI Magnesita N.V. is a global supplier of refractory products, systems, and services essential for high-temperature industrial processes.

9.419702069278545
RCP
RIT Capital Partners is a large British investment trust that invests in quoted securities and special situations, aiming to grow shareholders' wealth through a diversified, global portfolio.

16.658963347422315
ROR
Rotork plc is a UK-based company that designs and manufactures industrial flow control equipment, including actuators and related products, for various markets like oil & gas, water, and power.

23.329392999825398
RS1
RS Group plc is a global distributor of industrial and electrical products and service solutions, offering over 600,000 products to over 1 million customers.

10.202660745759538
RICA
Ruffer Investment Company is a British investment company focused on investments in internationally listed equities and related securities, aiming for positive annual returns and capital preservation.

27.47833671594

VEIL
Vietnam Enterprise Investments Ltd (VEIL) is a closed-ended equity mutual fund that primarily invests in listed and pre-IPO companies in Vietnam.

16.060293662942684
VOF
VinaCapital Vietnam Opp Fund Ord aims for medium to long-term returns by investing in Vietnam-based assets or companies with a majority of their operations in Vietnam.

43.11899324922991
VTY
Vistry Group PLC is a UK-based company focused on developing mixed-tenure homes and communities, operating through brands like Bovis Homes and Countryside Partnerships.

29.831144122106753
FAN
Volution Group plc is a leading supplier of ventilation products for residential and commercial buildings.

17.59380119981155
WPS
W.A.G Payment Solutions PLC (Eurowag) is a pan-European platform providing integrated payment and mobility solutions for the commercial road transport industry, including energy, toll payments, and related services.

19.262901102941868
WOSG
Watches of Switzerland Group PLC is an international retailer of luxur

In [ ]:
res.to_csv(list_name+'_annotation.csv', index=False)


import numpy as np
import pandas as pd
from scipy.stats import norm
import yfinance as yf
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

class VolatilityAnalyzer:
    def __init__(self, ticker, risk_free_rate=0.01):
        """
        Initialize the Volatility Analyzer
        
        Args:
            ticker (str): Stock ticker symbol
            risk_free_rate (float): Risk-free interest rate (default: 1%)
        """
        self.ticker = ticker
        self.risk_free_rate = risk_free_rate
        self.historical_data = None
        self.option_data = None
    
    def fetch_historical_data(self, period="1y"):
        """
        Fetch historical stock price data
        
        Args:
            period (str): Time period for historical data ('1y', '6mo', '3mo', etc.)
        
        Returns:
            pandas.DataFrame: Historical stock data
        """
        try:
            stock = yf.Ticker(self.ticker)
            self.historical_data = stock.history(period=period)
            return self.historical_data
        except Exception as e:
            print(f"Error fetching data for {self.ticker}: {e}")
            return None
    
    def calculate_realized_volatility(self, prices=None, window_days=21, method='rolling'):
        """
        Calculate realized volatility using historical stock prices
        
        Args:
            prices (pd.Series): Stock prices (uses fetched data if None)
            window_days (int): Rolling window size in days
            method (str): 'rolling' for rolling volatility, 'total' for entire period
        
        Returns:
            pd.Series or float: Realized volatility values
        """
        if prices is None:
            if self.historical_data is None:
                self.fetch_historical_data()
            prices = self.historical_data['Close']
        
        # Calculate log returns
        log_returns = np.log(prices / prices.shift(1)).dropna()
        
        if method == 'rolling':
            # Ensure window size doesn't exceed data length
            window_days = min(window_days, len(log_returns))
            if window_days < 2:
                return pd.Series([np.nan] * len(prices), index=prices.index)
            
            # Calculate rolling standard deviation
            rolling_std = log_returns.rolling(window=window_days, min_periods=2).std()
            # Annualize the volatility (252 trading days per year)
            realized_vol = rolling_std * np.sqrt(252)
            return realized_vol
        
        elif method == 'total':
            # Calculate volatility for entire period
            return log_returns.std() * np.sqrt(252)
    
    def black_scholes_call_price(self, S, K, T, r, sigma, q=0):
        """
        Calculate Black-Scholes call option price
        
        Args:
            S (float): Current stock price
            K (float): Strike price
            T (float): Time to expiration (in years)
            r (float): Risk-free rate
            sigma (float): Volatility
            q (float): Dividend yield (default: 0)
        
        Returns:
            float: Call option price
        """
        if T <= 0 or sigma <= 0:
            return max(S - K, 0)  # Intrinsic value
        
        d1 = (np.log(S / K) + (r - q + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
        d2 = d1 - sigma * np.sqrt(T)
        
        call_price = (S * np.exp(-q * T) * norm.cdf(d1) - 
                     K * np.exp(-r * T) * norm.cdf(d2))
        return call_price
    
    def black_scholes_put_price(self, S, K, T, r, sigma, q=0):
        """
        Calculate Black-Scholes put option price
        """
        if T <= 0 or sigma <= 0:
            return max(K - S, 0)  # Intrinsic value
        
        d1 = (np.log(S / K) + (r - q + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
        d2 = d1 - sigma * np.sqrt(T)
        
        put_price = (K * np.exp(-r * T) * norm.cdf(-d2) - 
                    S * np.exp(-q * T) * norm.cdf(-d1))
        return put_price
    
    def vega(self, S, K, T, r, sigma, q=0):
        """
        Calculate option Vega (sensitivity to volatility)
        """
        if T <= 0:
            return 0
        
        d1 = (np.log(S / K) + (r - q + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
        return S * np.exp(-q * T) * norm.pdf(d1) * np.sqrt(T)
    
    def calculate_implied_volatility(self, option_price, S, K, T, r, option_type='call', 
                                   q=0, initial_guess=0.2, tolerance=1e-6, max_iterations=100):
        """
        Calculate implied volatility using Newton-Raphson method
        
        Args:
            option_price (float): Market price of the option
            S (float): Current stock price
            K (float): Strike price
            T (float): Time to expiration (in years)
            r (float): Risk-free rate
            option_type (str): 'call' or 'put'
            q (float): Dividend yield
            initial_guess (float): Initial volatility guess
            tolerance (float): Convergence tolerance
            max_iterations (int): Maximum number of iterations
        
        Returns:
            float: Implied volatility
        """
        if T <= 0:
            return 0.0
        
        sigma = initial_guess
        
        for i in range(max_iterations):
            # Calculate theoretical option price
            if option_type.lower() == 'call':
                theoretical_price = self.black_scholes_call_price(S, K, T, r, sigma, q)
            else:
                theoretical_price = self.black_scholes_put_price(S, K, T, r, sigma, q)
            
            # Calculate price difference
            price_diff = theoretical_price - option_price
            
            # Check for convergence
            if abs(price_diff) < tolerance:
                return sigma
            
            # Calculate Vega
            option_vega = self.vega(S, K, T, r, sigma, q)
            
            # Avoid division by zero
            if abs(option_vega) < 1e-10:
                break
            
            # Newton-Raphson update
            sigma = sigma - price_diff / option_vega
            
            # Keep sigma positive and reasonable
            sigma = max(0.001, min(sigma, 5.0))
        
        return sigma
    
    def volatility_difference(self, realized_vol, implied_vol):
        """
        Calculate the difference between implied and realized volatilities
        
        Args:
            realized_vol (float): Realized volatility
            implied_vol (float): Implied volatility
        
        Returns:
            float: Volatility difference (IV - RV)
        """
        return implied_vol - realized_vol
    
    def analyze_volatility_spread(self, option_price, strike_price, expiration_days, 
                                option_type='call', window_days=21):
        """
        Complete analysis of volatility spread for a given option
        
        Args:
            option_price (float): Current market price of the option
            strike_price (float): Option strike price
            expiration_days (int): Days until expiration
            option_type (str): 'call' or 'put'
            window_days (int): Window for realized volatility calculation
        
        Returns:
            dict: Analysis results including RV, IV, and difference
        """
        # Fetch data if not available
        if self.historical_data is None:
            self.fetch_historical_data()
        
        # Get current stock price
        current_price = self.historical_data['Close'].iloc[-1]
        
        # Calculate time to expiration in years
        time_to_expiry = expiration_days / 365.0
        
        # Calculate realized volatility
        realized_vol = self.calculate_realized_volatility(window_days=window_days, method='rolling')
        current_realized_vol = realized_vol.iloc[-1]
        
        # Handle NaN values
        if pd.isna(current_realized_vol):
            # Use total period volatility if rolling window is too large
            current_realized_vol = self.calculate_realized_volatility(method='total')
        
        # Calculate implied volatility
        implied_vol = self.calculate_implied_volatility(
            option_price=option_price,
            S=current_price,
            K=strike_price,
            T=time_to_expiry,
            r=self.risk_free_rate,
            option_type=option_type
        )
        
        # Calculate volatility difference
        vol_diff = self.volatility_difference(current_realized_vol, implied_vol)
        
        # Calculate percentage difference
        vol_diff_pct = (vol_diff / current_realized_vol) * 100 if current_realized_vol != 0 else np.nan
        
        return {
            'ticker': self.ticker,
            'current_price': current_price,
            'strike_price': strike_price,
            'option_price': option_price,
            'option_type': option_type,
            'days_to_expiry': expiration_days,
            'realized_volatility': current_realized_vol,
            'implied_volatility': implied_vol,
            'volatility_difference': vol_diff,
            'volatility_difference_pct': vol_diff_pct,
            'analysis_date': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        }

# Example usage function
def example_usage():
    """
    Example of how to use the VolatilityAnalyzer
    """
    # Initialize analyzer for Apple stock
    analyzer = VolatilityAnalyzer('AAPL', risk_free_rate=0.02)
    
    # Example analysis for a call option
    results = analyzer.analyze_volatility_spread(
        option_price=5.50,        # Current option price
        strike_price=150,         # Strike price
        expiration_days=30,       # Days until expiration
        option_type='call',       # Option type
        window_days=21           # Rolling window for RV
    )
    
    return results

# Run example if script is executed directly
if __name__ == "__main__":
    results = example_usage()
    print("Volatility Analysis Results:")
    for key, value in results.items():
        if isinstance(value, float):
            print(f"{key}: {value:.4f}")
        else:
            print(f"{key}: {value}")
